In [24]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129556")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129556
Azure region: southcentralus
Subscription id: 4910dccd-0348-46c4-a51f-d8c85e078b14
Resource group: aml-quickstarts-129556


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "D2V2Cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.5,5),
        "--max_iter": choice(20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200)
    }
)

# Specify a Policy
# If, after 10 intervals, the current run has an absolute distance of > 0.2 from the current best run, terminate it.
policy = BanditPolicy(evaluation_interval=10,
                      slack_amount=0.2,
                      delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(".", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
                                     estimator=est,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [25]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

os.makedirs("outputs", exist_ok=True)

joblib.dump(value=best_run.id, filename="outputs/model.joblib")

['--C', '1.0306977840606755', '--max_iter', '70']


['outputs/model.joblib']

In [26]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [30]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [39]:
from azureml.train.automl import AutoMLConfig
import pandas as pd
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=pd.concat([x,y]),
    validation_size=0.2,
    label_column_name="poutcome",
    n_cross_validations=5)

In [40]:
# Submit your automl run

automl_run = exp.submit(automl_config,ws)
RunDetails(automl_run).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [ ]:
# Retrieve and save your best automl model.

automl_best_run = automl_run.get_best_run_by_primary_metric()
print(automl_best_run.get_details()['runDefinition']['arguments'])

os.makedirs("outputs", exist_ok=True)

joblib.dump(value=best_run.id, filename="outputs/automl_model.joblib")